# 📄 DeepLog: Anomaly Detection and Diagnosis from System Logs
> ACM CCS 2017 | Min Du, Feifei Li, Guineng Zheng, Vivek Srikumar  
> [논문 링크](https://users.cs.utah.edu/~lifeifei/papers/deeplog.pdf)

---

## ✅ Day 1 – Abstract & Introduction

---

### 📌 연구 배경과 동기
- **현대 시스템의 로그 문제**  
  대규모 분산 시스템(Hadoop, Spark 등)은 초당 수천 건 이상의 로그를 생성한다. 이 로그는 시스템의 상태와 이상 행동을 파악할 수 있는 핵심 단서이지만, 데이터의 양과 복잡성으로 인해 사람이 직접 분석하기 어렵다.
  

- **기존 접근의 한계**  
  - 규칙 기반 탐지 방식은 미리 정의된 패턴에만 반응하므로 새로운 유형의 이상(New Anomaly)을 탐지하지 못한다.  
  - PCA, SVM 등 통계적·전통적 머신러닝 기법은 로그의 **순차적 의존성(sequence dependency)** 을 반영하지 못한다.  


- **핵심 연구 질문**  
  “로그를 언어 시퀀스로 모델링하여,  
  다음에 발생할 이벤트를 예측함으로써 이상을 탐지할 수 있을까?”
  
<br>

### 📌 핵심 아이디어
- 로그를 **문장(Sequence)** 으로, 각 로그 이벤트를 **단어(Token)** 로 간주한다.  
- LSTM(Long Short-Term Memory) 구조를 이용해 **정상 시퀀스의 다음 이벤트를 예측**한다.  
- 예측 결과와 실제 이벤트가 일치하지 않으면 이를 **이상(Anomaly)** 으로 판단한다.  
- 학습은 정상 로그만으로 이루어지며, **One-class Supervised Learning** 형태로 동작한다.

<br>

### 📌 주요 기여
1. **시퀀스 기반 이상탐지 프레임워크 제안**  
   로그를 시계열 문맥으로 해석하고 LSTM으로 다음 이벤트를 예측하는 방법을 제시했다.  
2. **자동 진단(Diagnosis) 기능 제공** 이상이 발생하면 관련 로그 시퀀스를 역추적하여 원인을 분석할 수 있다.  
3. **성능 향상 입증**  
   PCA, Invariant Mining 등 기존 방식보다 정밀도와 재현율이 모두 향상됨을 보였다.  

<br>

### 📌 핵심 인사이트
- 로그는 단순한 시스템 출력이 아니라, **시스템의 언어**이다.  
- DeepLog는 이 언어의 패턴을 학습하여 시스템의 정상 행동을 이해하고,그로부터 벗어나는 패턴을 이상으로 탐지한다.  
- 이는 규칙을 정의하는 기존 관점에서 벗어나,**정상의 맥락(Context)을 학습하는 관제 방식**으로의 전환을 의미한다.  

<br>

### 🧠 개인적 해석
- DeepLog는 “정상을 정의하면 이상은 자연히 드러난다”는 철학을 잘 보여준다.  
- SOC 환경에서 딥러닝을 적용한 초기 사례로서, **‘로그 = 언어’라는 개념적 전환점**을 마련했다.  
- 이러한 시퀀스 기반 접근은 이후 Transformer 구조를 활용한 LogBERT, LogGPT 등의 후속 연구로 발전하는 출발점이 되었다.

---

## ✅ Day 2 – Related Work & Framework Overview

---

### 📌 관련 연구 (Related Work)

- **규칙 기반 / 통계 기반 접근의 한계**
  - 전통적으로 로그 이상탐지는 **정의된 규칙(rule)**, **정상 분포 기반 모델**, 또는 **PCA 기반 차원축소** 등을 활용했다.  
  - 하지만 이런 방법들은 **패턴 변화에 취약**하고, **로그 간 시간적·의존 관계**를 고려하지 못한다.
  - 예: `Event A → Event B → Event C` 가 정상이라면, `Event A → Event D` 같은 새로운 조합은 감지 불가능하다.


- **머신러닝 기반 접근**
  - SVM, Isolation Forest 등 일부 ML 기법도 사용되었지만,  
    로그가 갖는 **순차적 구조(sequence dependency)** 를 명시적으로 모델링하지 못한다.
  - 결국 로그는 “독립된 점들의 집합”으로 처리되어, 맥락 이해가 불가능했다.


- **DeepLog의 새로운 시도**
  - DeepLog는 로그를 **자연어 문장처럼 모델링**하여,  
    **다음 이벤트 예측(next event prediction)** 문제로 변환한다.  
  - 이를 통해 시간 순서, 이벤트 간 전이 패턴, 반복 구조 등을 학습할 수 있다.
  
<br>

### 📌 전체 구조 (Framework Overview)


#### 1️⃣ 로그 전처리 단계 (Log Parsing)

<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
INFO Block 1234 received from node 10.1.1.5  
→ Block <*> received from node <*>  
→ Event ID = 45  
</div>


- 로그는 원래 **비정형 문자열**이므로, 이를 **Log Key** 단위로 변환한다.  
- 위와 같은 과정을 통해 로그를 **토큰화된 시퀀스(event ID sequence)** 로 표현할 수 있다.  
- 일반적으로 **Drain**, **Spell** 같은 로그 파서(parser)를 사용해 정규화한다.  


#### 2️⃣ 시퀀스 모델 학습 (Sequence Modeling)

- DeepLog는 로그 시퀀스를 **LSTM(Long Short-Term Memory)** 기반 모델에 입력하여 다음에 발생할 이벤트를 예측한다.  
- 모델은 최근 **W개의 이벤트 윈도우(window)** 를 관찰하고, 그 문맥으로부터 다음 이벤트를 예측한다.

$$
P(x_{t+1} | x_t, x_{t-1}, \ldots, x_{t-W+1})
$$

- 학습은 **정상 로그 데이터만**을 이용하며,이는 비정상 레이블이 없는 환경에서의 **One-class Supervised Training** 형태이다.  


#### 3️⃣ 이상 탐지 (Anomaly Detection)

- 실제 로그 스트림에서 다음 이벤트가 발생했을 때,  
  모델이 예측한 상위 Top-k 이벤트 목록에 없다면 → **이상(Anomaly)** 으로 판단한다.
  
<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
Input sequence: [E1, E2, E3, E4]  
Predicted next events: [E5, E6, E7]  
Actual next event: E9  
→ E9 ∉ Top-k → **Anomaly Detected** 
</div>

- 이런 방식은 새로운 이벤트 조합이나 이전에 없던 패턴도 감지할 수 있게 한다.  


#### 4️⃣ 진단 단계 (Diagnosis)

- 이상이 감지되면, DeepLog는 해당 시퀀스의 **hidden state 변화**를 분석하여 이상이 발생한 위치(원인 이벤트)를 추정한다.  
- 즉, 단순 탐지뿐 아니라 **원인 규명(root-cause analysis)** 도 수행할 수 있다. 

<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
예시:  
정상: A → B → C → D  
이상:  A → B → X → D
</div>

→ Hidden state 변화가 X에서 급격함 → X가 root cause로 추정 

- 실제 논문에서는 LSTM의 내부 상태를 시각화하여 정상/비정상 시퀀스의 차이를 명확히 보여주었다.  

<br>

### 📌 핵심 인사이트

- 로그를 **시간 순서가 있는 언어 시퀀스**로 해석한 것이 핵심 혁신이다.  
- LSTM은 시퀀스 내의 **장기 의존성(Long-term dependency)** 을 학습하여 시스템의 “정상 동작 패턴”을 내재적으로 파악한다.  
- 이상 탐지는 더 이상 “규칙 위반”이 아니라, **예측 불일치(prediction inconsistency)** 로 정의된다.  

<br>

### 🧠 개인적 해석

- DeepLog의 파이프라인은 NLP의 “다음 단어 예측(next-token prediction)” 구조와 동일하다.  
- 정상 로그만으로 학습한다는 점에서, **One-Class Learning** 혹은 **Self-Supervised Prediction**의 초기 형태라 할 수 있다.  
- “Top-k 예측 안에 없으면 이상”이라는 간단한 규칙은 실제 시스템 환경에서도 구현이 간단하고 효과적이다.

---
## ✅ Day 3 – Model Details & Training Process
---

### 📌 1️⃣ 입력 데이터 구조 (Input Representation)

- DeepLog는 로그를 **이벤트 ID 시퀀스**로 변환한 후, 최근 `w`개의 로그를 입력으로 사용한다.  
- 각 로그 이벤트는 **임베딩 벡터(embedding vector)** 로 매핑된다.  
  (단순한 정수 ID가 아니라, 이벤트 간 유사도를 학습할 수 있는 벡터 표현이다.)  

입력 시퀀스 예시:  
<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
x_t = [e_{t-w+1}, e_{t-w+2}, ..., e_t]
</div>
- 임베딩 차원(dimension)은 약 128~256 수준으로 설정하며, 학습 중 자동으로 업데이트된다.  
- 입력 윈도우 크기(`w`)는 로그 패턴의 길이에 따라 조정하며, 논문에서는 `w = 10`이 최적이었다.

---

### 📌 2️⃣ LSTM 네트워크 구조 (Network Architecture)

- LSTM은 순차 데이터의 시간적 의존성을 학습하기 위한 신경망이다.  
- 각 시간 단계에서, LSTM은 현재 이벤트의 임베딩(`e_t`)과 이전 hidden state(`h_{t-1}`)를 받아 새로운 상태(`h_t`)를 계산한다.

간단한 형태로 표현하면:
<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
h_t = LSTM(e_t, h_{t-1})
</div>

- 마지막 hidden state `h_t`는 fully connected layer에 전달되어 다음 이벤트의 확률 분포를 예측한다.  

예측 단계:
<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">  
P(x_{t+1}) = Softmax(W * h_t + b)
</div>

- 여기서 `W`, `b`는 학습 가능한 파라미터이며, Softmax를 통해 모든 이벤트 ID에 대한 확률을 구한다.

---

### 📌 3️⃣ 학습 과정 (Training Phase)

- 학습 데이터는 **정상 로그 시퀀스만**으로 구성된다.  
- 학습 목표는 “다음 이벤트를 맞추는 것”이며, 손실 함수는 **Cross-Entropy Loss**이다.

<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;"> 
Loss = - Σ ( y_i * log(ŷ_i) )
</div>


- `y_i`: 실제 다음 이벤트의 one-hot 벡터  
- `ŷ_i`: 모델이 예측한 확률 분포  

**학습 설정 (논문 기준):**
- Optimizer: Adam  
- Learning rate: 0.001  
- Batch size: 128  
- Epochs: 약 50~100  
- Early Stopping 적용  

---

### 📌 4️⃣ 이상 탐지 로직 (Detection Logic)

- 예측 시, 모델은 다음 이벤트의 확률 상위 `K`개를 계산한다.  
- 실제 발생한 이벤트가 그 안에 포함되지 않으면 → **이상(Anomaly)** 으로 판단한다.

예시:  
<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;"> 
입력 시퀀스: [E1, E2, E3, E4]
모델 예측 Top-3: [E5, E6, E7]
실제 이벤트: E9
→ E9 not in Top-3 → Anomaly Detected
</div>

- `K` 값은 정밀도(Precision)와 재현율(Recall)의 균형을 조정하는 파라미터이며,  
  논문에서는 `K = 9`일 때 가장 우수한 성능을 보였다.

---

### 📌 5️⃣ 이상 진단 (Anomaly Diagnosis)

- LSTM의 hidden state 변화량을 분석하면 **이상의 원인 이벤트(root-cause)** 를 추정할 수 있다.  
- hidden state 간 거리(예: cosine distance)가 갑자기 커지는 지점이 비정상 행동의 원인이다.

예시:  
<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;"> 
정상 시퀀스: A → B → C → D
이상 시퀀스: A → B → X → D
</div>
→ hidden state 변화가 X에서 급격함 → X가 root cause로 추정됨

---

### 📊 주요 하이퍼파라미터 요약

| 파라미터 | 설명 | 값 (논문 기준) |
|-----------|------|----------------|
| Window size (w) | 입력 시퀀스 길이 | 10 |
| Hidden size | LSTM 은닉 차원 | 128 |
| Embedding size | 이벤트 임베딩 차원 | 128 |
| Batch size | 학습 배치 크기 | 128 |
| Learning rate | 학습률 | 0.001 |
| Optimizer | 가중치 업데이트 방식 | Adam |
| Top-K | 이상 탐지 기준 | 9 |

---

### 💡 핵심 인사이트

- DeepLog는 로그를 언어처럼 다루며, 정상 동작의 문맥을 학습한다.  
- 정상 로그만으로 시스템의 행동 패턴을 충분히 학습할 수 있음을 보여준다.  
- Hidden state를 이용한 진단 기능은 단순 탐지를 넘어,  
  **“왜 이상이 발생했는가”** 를 설명할 수 있게 한다.

---

### 🧠 개인적 해석

- DeepLog는 “시스템 로그의 언어모델(System Log Language Model)” 개념의 출발점이다.  
- 정상 로그 기반 학습은 실제 SOC 환경(이상 레이블 부족)에 매우 현실적이다.  
- hidden state 분석은 보안 관제에서 이상 원인(예: 특정 이벤트 조합)을 시각적으로 파악하는 데 유용하다.  
- 오늘날 LogBERT, LogGPT 등의 Transformer 기반 모델은 모두 이 구조를 확장한 형태라 볼 수 있다.

---
## ✅ Day 4 – Experiments & Results
---

### 📌 1️⃣ 실험 환경 (Experimental Setup)

- **데이터셋 구성**
  - **HDFS (Hadoop Distributed File System)**  
    → 약 11M lines, 48 event types, 정상 로그만 포함  
  - **BGL (BlueGene/L Supercomputer Logs)**  
    → 약 4M lines, 정상/이상 로그 포함  
  - **OpenStack Logs**  
    → 클라우드 관리 플랫폼 환경에서 수집된 로그 데이터  

- **사전 처리 과정**
  - 로그 파싱은 **Drain 알고리즘**을 사용하여 템플릿(event ID)으로 변환함  
  - 이후 각 로그는 **정수 인덱스 → 임베딩 벡터**로 매핑됨  
  - 학습 데이터는 정상 로그 시퀀스로만 구성  

- **학습 환경**
  - Framework: TensorFlow  
  - Optimizer: Adam  
  - GPU: NVIDIA GTX 1080Ti  
  - Epoch: 50~100 (Early Stopping 적용)  

---

### 📌 2️⃣ 비교 대상 (Baselines)

DeepLog는 다음과 같은 전통적 이상 탐지 기법들과 비교되었다.

| 모델 | 접근 방식 | 특징 |
|------|------------|------|
| **PCA** | 통계 기반 | 로그의 주요 성분으로 이상 탐지 |
| **Invariant Mining** | 규칙 기반 | 이벤트 간 불변관계(invariants) 학습 |
| **Isolation Forest** | 트리 기반 | 고립 정도로 이상 판단 |
| **Logistic Regression** | 지도학습 기반 | 라벨이 있는 로그에서 학습 필요 |
| **DeepLog (제안)** | 순차 학습 기반 | 로그 시퀀스의 문맥을 모델링 |

> DeepLog는 시퀀스의 순서를 학습하기 때문에, 단일 이벤트 기반의 기존 방식보다 변화 탐지에 강하다.

---

### 📌 3️⃣ 평가 지표 (Evaluation Metrics)

- **Precision (정밀도)**  
  예측한 이상 중 실제 이상인 비율  
- **Recall (재현율)**  
  실제 이상 중 탐지된 비율  
- **F1-score**  
  Precision과 Recall의 조화 평균  
- **False Positive Rate (FPR)**  
  정상 로그가 오탐된 비율  
- **Detection Delay**  
  이상 발생 후 탐지까지 걸린 시간  

> 논문에서는 주로 F1-score를 중심으로 성능을 비교하였다.

---

### 📌 4️⃣ 실험 결과 요약 (Main Results)

- DeepLog는 **모든 데이터셋에서 F1-score 기준 최고 성능**을 달성하였다.
- HDFS 데이터셋 기준, 기존 PCA 대비 **약 15% 이상 성능 향상**.
- BGL, OpenStack 환경에서도 높은 **재현율(Recall)** 을 유지.
- `Top-K`가 커질수록 이상 탐지율은 향상되지만, 오탐 가능성도 증가함.

<div style="background-color:#f4f9ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
예시 결과 요약:  
F1-score(%) 기준  
PCA: 78.2 → DeepLog: **93.1**  
Isolation Forest: 81.4 → DeepLog: **92.5**
</div>

> 특히 DeepLog는 **복잡한 이벤트 간 순차 의존성**을 모델링할 수 있어,  
> 패턴이 긴 로그 시퀀스에서 높은 탐지 성능을 보였다.

---

### 📌 5️⃣ 파라미터 분석 (Hyperparameter Analysis)

- **Window Size (w)**  
  너무 작으면 패턴이 충분히 반영되지 않고, 너무 크면 학습이 불안정  
  → 최적값: `w = 10`

- **Top-K**  
  `K`가 커질수록 Recall ↑, Precision ↓  
  → `K = 9`에서 F1-score가 가장 높음

- **Hidden Size, Embedding Size**  
  128~256 차원 구간에서 안정적인 성능 유지  

<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
실험적으로 확인된 최적 조합:  
w = 10, Hidden = 128, Embedding = 128, Top-K = 9
</div>

---

### 📌 6️⃣ Root-Cause Diagnosis 결과

- DeepLog는 단순한 탐지를 넘어, **이상 원인 이벤트(root cause)** 를 추적 가능함.  
- LSTM hidden state 간의 **거리(Cosine Distance)** 를 분석하여 이상 구간을 식별.  
- 정상 시퀀스에서는 hidden state 변화가 점진적이지만,  
  이상 시퀀스에서는 특정 시점에서 급격한 변화가 발생한다.

<div style="background-color:#eef6ff; padding:12px 15px; border-left:5px solid #4A90E2; border-radius:6px;">
예시:  
정상: A → B → C → D  
이상: A → B → X → D  
→ X에서 hidden state 변화량 급증 → X가 root cause로 추정
</div>

---

### 📌 7️⃣ 실무 적용 가능성 (Practical Implications)

- **SOC/SIEM 환경**에서의 활용  
  - 비정상 시스템 동작 조기 탐지  
  - 로그 패턴 기반 경보(Alarm) 자동화  
- **레이블이 부족한 환경**에서도 적용 가능 (Unsupervised 학습 구조)  
- 단, 새로운 시스템에서는 초기 정상 로그 확보가 필요함 (“Cold Start” 문제)

---

### 💡 핵심 인사이트

- DeepLog는 단순한 이상 탐지기가 아니라,  
  **“로그를 언어처럼 해석하는 모델(Language Model for Logs)”** 로서의 가능성을 보여줌.  
- 시퀀스 문맥을 이해함으로써 **규칙 기반 접근의 한계를 극복**하였다.  
- Root-Cause 분석 기능은 이후의 **Explainable AI(LogBERT, LogGPT 등)** 로 발전하는 기초가 되었다.

---

### 🧠 개인적 해석

- DeepLog의 가장 큰 강점은 **정상 로그만으로 시스템 행동을 학습**할 수 있다는 점이다.  
- SOC 환경에서 실제 로그 레이블은 거의 없으므로, 이 접근은 현실적으로 매우 적합하다.  
- Root-Cause Localization은 실제 이상 대응(Incident Response)에도 유용하며,  
  추후 Transformer 기반 모델(LogBERT, LogGPT)은 이 구조를 그대로 확장하였다.  
- 결국 DeepLog는 “보안 로그 언어모델”의 출발점이라 할 수 있다.

---

---
## ✅ Day 5 – Discussion & Future Work
---

### 📌 1️⃣ 논의 및 한계점 (Discussion & Limitations)

- **이상 로그의 레이블 부족 문제**  
  DeepLog는 정상 로그만으로 학습하기 때문에 레이블링 노력이 크게 줄지만,  
  “완전한 정상 데이터셋”을 보장하기 어렵다는 한계가 있다.  
  실제 운영 환경에서는 정상 로그 속에 경미한 이상이 섞여 있을 수 있다.

- **Cold Start 문제**  
  새로운 시스템이나 애플리케이션에서는 초기 로그가 부족하여 모델 학습이 어렵다.  
  즉, 충분한 정상 시퀀스를 수집하기 전까지 탐지 성능이 낮을 수 있다.

- **고정된 시퀀스 길이 제약**  
  입력 윈도우 `w` 길이가 고정되어 있기 때문에 다양한 패턴 길이를 동시에 처리하기 어렵다.  
  긴 종속성을 가진 로그 패턴에서는 성능 저하가 발생할 수 있다.

- **실시간 탐지 성능 제한**  
  LSTM 기반 모델은 순차 연산이 필수적이라 대규모 스트리밍 로그를 실시간으로 처리할 때 병목이 발생할 수 있다.

- **해석 가능성 (Explainability)**  
  hidden state 분석으로 이유를 부분적으로 설명할 수 있지만,  
  내부 가중치나 의사결정 경로가 직관적으로 드러나지는 않는다.  
  완전한 설명가능 모델로 보기는 어렵다.

---

### 📌 2️⃣ 향후 연구 방향 (Future Work & Extensions)

- **Transformer 기반 구조로의 확장**  
  RNN/LSTM 의 한계를 극복하기 위해 이후 LogBERT, LogGPT 등이 출현하였다.  
  이들은 Self-Attention 으로 긴 의존 관계를 효율적으로 모델링하며,  
  DeepLog의 시퀀스 예측 아이디어를 Transformer 프레임워크로 일반화했다.

- **Pre-training 및 Fine-tuning**  
  대규모 로그 코퍼스에서 사전학습 (pre-training) 후 도메인별 미세조정 (fine-tuning) 을 수행하는 방식이 주류가 되었다.  
  이를 통해 데이터셋 별 Cold Start 문제를 완화할 수 있다.

- **멀티모달 로그 분석**  
  단순한 텍스트 로그 외에도 시스템 메트릭(CPU, RAM), 네트워크 패킷, 보안 이벤트를 결합한  
  **다중 소스 분석(Multimodal Anomaly Detection)** 으로 확장 가능하다.

- **온라인 학습(Online Learning)**  
  로그 스트림 환경에 맞게 모델 가중치를 실시간으로 업데이트하는 기법이 필요하다.  
  이를 통해 시간 흐름에 따른 시스템 패턴 변화를 적응적으로 반영할 수 있다.

- **설명 가능한 AI 접근 (XAI for Logs)**  
  LIME, SHAP 등을 활용해 이상 탐지 결과를 시각화하고,  
  “왜 이 이벤트가 이상으로 판단되었는가?” 를 투명하게 제시하는 연구가 진행 중이다.

---

### 📌 3️⃣ 보안 관제 측면의 의의 (Implications for Cybersecurity)

- **SOC 및 SIEM 자동화**  
  DeepLog 구조는 보안 관제 센터(SOC)에서 로그 기반 이상 탐지를 자동화할 수 있는 기초 틀을 제공한다.  
  정해진 규칙(rule) 없이 학습된 모델이 새로운 공격 패턴을 포착할 수 있다는 점이 핵심이다.

- **사고 원인 분석 및 대응**  
  hidden state 변화 기반 root-cause 추론은 사고 이후 로그 포렌식 (log forensics) 에 유용하다.  
  DeepLog 는 이상 발생 시점뿐 아니라 “어떤 이벤트 조합이 문제를 일으켰는가” 까지 파악할 수 있다.

- **모델 통합 가능성**  
  최근에는 IDS/IPS, EDR 등 다른 보안 시스템에서 수집한 로그를 통합하여  
  DeepLog 유형의 모델과 결합하는 연구가 진행되고 있다.

---

### 💡 핵심 인사이트

- DeepLog는 “로그 언어모델”이라는 새로운 패러다임을 연 논문이다.  
- 규칙 기반 탐지 → 시퀀스 학습 → 언어모델 탐지로 발전하는 기술적 전환점이었다.  
- 이후의 Transformer 기반 로그 분석은 모두 DeepLog의 사상을 계승하고 있다.  
- **‘Anomaly Detection → Anomaly Explanation’** 으로의 전환을 이끈 초기 연구로서 가치가 크다.

---

### 🧠 개인적 해석

- DeepLog의 등장은 SOC 자동화의 시발점이라 볼 수 있다.  
  단순 탐지를 넘어 “이상 상황을 설명할 수 있는 보안 AI” 의 기반을 닦았다.  
- 오늘날 LogBERT 나 GPT-for-Logs 와 같은 모델들은  
  DeepLog의 LSTM 언어모델 구조를 Transformer로 치환한 형태라고 할 수 있다.  
- 향후 보안 AI 연구는 이상 탐지 정확도뿐 아니라,  
  **해석 가능성(Explainability)** 과 **자동 대응(Automated Response)** 으로 진화할 것이다.  
- DeepLog 는 그 첫 걸음이자, SOC 지능화의 역사적 시작점으로 기록될 만하다.

---
